In [28]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score

In [1]:
DATA_DIR = 'C:/Users/Akarshjayanth.M.Naik/Desktop/my_fruit_checker/data/testing'  # path of the test data
IMG_SIZE = 50
MODEL_PATH = 'fruitquality-structured-cnn.h5'

In [30]:

# Detect Fruit Types

FRUIT_TYPES = []
for item in os.listdir(DATA_DIR):
    fruit_path = os.path.join(DATA_DIR, item)
    if os.path.isdir(fruit_path):
        if os.path.isdir(os.path.join(fruit_path, 'fresh')) and os.path.isdir(os.path.join(fruit_path, 'rotten')):
            FRUIT_TYPES.append(item)

In [31]:

# Recreate CATEGORIES and Label Map

CATEGORIES = []
label_map = {}  # e.g., {'fresh_apple': 0, 'rotten_apple': 1, ...}
index = 0
for fruit in FRUIT_TYPES:
    for quality in ['fresh', 'rotten']:
        cat = f"{quality}_{fruit}"
        CATEGORIES.append(cat)
        label_map[cat] = index
        index += 1

In [32]:

# Load Test Data

def load_test_data():
    test_data = []
    y_true = []

    # Define label_map based on CATEGORIES list from training
    label_map = {category: index for index, category in enumerate(CATEGORIES)}

    for fruit in FRUIT_TYPES:
        for quality in ['fresh', 'rotten']:
            combined_category_name = f"{quality}_{fruit}" # Construct combined category name
            if combined_category_name not in label_map: # Check if combined category is in label_map
                print(f"Warning: Combined category '{combined_category_name}' not found in label map. Skipping.")
                continue # Skip if the category is not in the label map

            label = label_map[combined_category_name] # Use combined category name to get label
            path = os.path.join(DATA_DIR, fruit, quality) # Use DATA_DIR (ensure it's set to test data dir if needed)
            for img in os.listdir(path):
                try:
                    img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                    if img_array is None:
                        print(f"Warning: Could not read image {img} in {combined_category_name}. Skipping.")
                        continue
                    img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                    test_data.append(img_array)
                    y_true.append(label)
                except Exception as e:
                    print(f"Error with image {img} in {combined_category_name}: {e}")
                    continue
    return np.array(test_data), np.array(y_true)


# --- Load the test data ---
X_test, y_test = load_test_data()

X_test = X_test.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X_test = X_test / 255.0  # Normalize

print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_test: (4362, 50, 50, 1)
Shape of y_test: (4362,)


In [33]:

# Load Model and Predict
model = load_model(MODEL_PATH)
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)


# Evaluate Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\n✅ Test Accuracy: {accuracy * 100:.2f}%")


137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step

✅ Test Accuracy: 95.71%
